In [ ]:
#Invite everyone to the party
import pandas as pd
import numpy as np
import datetime as DT
import hmac
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
%matplotlib inline
from sqlalchemy import create_engine
import pyodbc 
from geopy import Point
from geopy.distance import distance



In [ ]:

#Connect to the database -- to be changed depending on configuration of the database

cnxn = pyodbc.connect(
    Trusted_Connection='Yes',
    Driver='{SQL Server Native Client 11.0}',
    Server='SERVER_NAME',
    Database='DATABASE_NAME'
)

cursor = cnxn.cursor()


In [ ]:
SQL_RELATIONSHIP_TABLES = """	---Create Source
	select [Customer_id]
      ,[Full_name]
      ,[Customer_type]
      ,[Is_client_flg]
      ,[Is_referal]
      ,[Ref_Customer_id]
      ,[Country]
      ,[Region]
      ,[City]
      ,[Street]
      ,[Street_nr]
      ,[Activation_date]
      ,[Contact_phone]
      ,[Contact_email]
      ,[Acc_mgr_id]
      ,[Acc_mgr_name]

	  into Customer_Source

	  from Customer

	  where Customer_id in (select Ref_Customer_id from Customer where Ref_Customer_id is not null)


	---Create destination table  

	select [Customer_id]
      ,[Full_name]
      ,[Customer_type]
      ,[Is_client_flg]
      ,[Is_referal]
      ,[Ref_Customer_id]
      ,[Country]
      ,[Region]
      ,[City]
      ,[Street]
      ,[Street_nr]
      ,[Activation_date]
      ,[Contact_phone]
      ,[Contact_email]
      ,[Acc_mgr_id]
      ,[Acc_mgr_name]
	  into Customer_Destination

	  from Customer

	  where Ref_Customer_id is not null


---Create bridge table

 select [Customer_id],[Ref_customer_id] into Source_Destination_r

	  from Customer

	  where ref_customer_id is not null

---Create leads table

select [Customer_id]
      ,[Full_name]
      ,[Customer_type]
      ,[Is_client_flg]
      ,[Is_referal]
      ,[Ref_Customer_id]
      ,[Country]
      ,[Region]
      ,[City]
      ,[Street]
      ,[Street_nr]
      ,[Activation_date]
      ,[Contact_phone]
      ,[Contact_email]
      ,[Acc_mgr_id]
      ,[Acc_mgr_name] into [dbo].[Customer_Leads] 
from [dbo].[Customer] where Customer_type = 'Lead'




---Create full table

select 	s.[Customer_id] as Source_Customer_id
,	s.[Full_name] as Source_Full_name
,	s.[Customer_type] as Source_Customer_type
,	s.[Is_client_flg] as Source_Is_client_flg
,	s.[Country] as Source_Country
,	s.[Region] as Source_Region
,	s.[City] as Source_City
,	s.[Street] as Source_Street
,	s.[Street_nr] as Source_Street_nr
,	s.[Activation_date] as Source_Activation_date
,	s.[Contact_phone] as Source_Contact_phone
,	s.[Contact_email] as Source_Contact_email
,	s.[Acc_mgr_id] as Source_Acc_mgr_id
,	s.[Acc_mgr_name] as Source_Acc_mgr_name	
  ,	d.[Customer_id] as Destination_Customer_id
,	d.[Full_name] as Destination_Full_name
,	d.[Customer_type] as Destination_Customer_type
,	d.[Is_client_flg] as Destination_Is_client_flg
,	d.[Country] as Destination_Country
,	d.[Region] as Destination_Region
,	d.[City] as Destination_City
,	d.[Street] as Destination_Street
,	d.[Street_nr] as Destination_Street_nr
,	d.[Activation_date] as Destination_Activation_date
,	d.[Contact_phone] as Destination_Contact_phone
,	d.[Contact_email] as Destination_Contact_email
,	d.[Acc_mgr_id] as Destination_Acc_mgr_id
,	d.[Acc_mgr_name] as Destination_Acc_mgr_name

into Source_Destination_Full

from [dbo].[Source_Destination_r] r
left join [dbo].[Customer_Source] s
on r.Ref_Customer_id = s.Customer_id
left join [dbo].[Customer_Destination] d
on r.Customer_id = d.Customer_id """

In [ ]:
##See create relationship table SQL

###GEOCODE RELATIONSHIP TABLE
#Define dataframe for customer relationship
sql_customer_relationship = """SELECT [Source_Customer_id]
      ,[Source_Full_name]
      ,[Source_Customer_type]
      ,[Source_Is_client_flg]
      ,[Source_Country]
      ,[Source_Region]
      ,[Source_City]
      ,[Source_Street]
      ,[Source_Street_nr]
      ,[Source_Activation_date]
      ,[Source_Contact_phone]
      ,[Source_Contact_email]
      ,[Source_Acc_mgr_id]
      ,[Source_Acc_mgr_name]
      ,[Destination_Customer_id]
      ,[Destination_Full_name]
      ,[Destination_Customer_type]
      ,[Destination_Is_client_flg]
      ,[Destination_Country]
      ,[Destination_Region]
      ,[Destination_City]
      ,[Destination_Street]
      ,[Destination_Street_nr]
      ,[Destination_Activation_date]
      ,[Destination_Contact_phone]
      ,[Destination_Contact_email]
      ,[Destination_Acc_mgr_id]
      ,[Destination_Acc_mgr_name]
  FROM [dbo].[Source_Destination_Full]"""


customer_relationship = pd.read_sql(sql_customer_relationship,cnxn)
customer_relationship.head()


In [ ]:

#Define full address destination

customer_relationship['Destination_full_address'] = customer_relationship['Destination_Country'] + ","+ customer_relationship['Destination_Region'] + ","+ customer_relationship['Destination_City'] + ","+ customer_relationship['Destination_Street'] + ","+ customer_relationship['Destination_Street_nr'] + " " 

##Define full address source
customer_relationship['Source_full_address'] = customer_relationship['Source_Country'] + ","+ customer_relationship['Source_Region'] + ","+ customer_relationship['Source_City'] + ","+ customer_relationship['Source_Street'] + ","+ customer_relationship['Source_Street_nr'] + " " 

customer_relationship.head()



In [ ]:

#Cleanup the data
customer_relationship = customer_relationship[customer_relationship['Source_full_address'].notna()]
customer_relationship = customer_relationship[customer_relationship['Destination_full_address'].notna()]

customer_relationship.head()


In [ ]:

###Geocode Source--takes a while
geolocator = Nominatim(user_agent="myApp")

for i in customer_relationship.index:
    try:
        #tries fetch address from geopy
        location = geolocator.geocode(customer_relationship['Source_full_address'][i])
        
        #append lat/long to column using dataframe location
        customer_relationship.loc[i,'source_lat'] = location.latitude
        customer_relationship.loc[i,'source_long'] = location.longitude
        customer_relationship.loc[i,'source_address'] = location.address
    except:
        #catches exception for the case where no value is returned
        #appends null value to column
        customer_relationship.loc[i,'source_lat'] = ""
        customer_relationship.loc[i,'source_long'] = ""
        customer_relationship.loc[i,'source_address'] = ""

#print first rows as sample
customer_relationship.head()


In [ ]:

#Geocode destination---takes a while
geolocator = Nominatim(user_agent="myApp")

for i in customer_relationship.index:
    try:
        #tries fetch address from geopy
        location = geolocator.geocode(customer_relationship['Destination_full_address'][i])
        
        #append lat/long to column using dataframe location
        customer_relationship.loc[i,'destination_lat'] = location.latitude
        customer_relationship.loc[i,'destination_long'] = location.longitude
        customer_relationship.loc[i,'destination_address'] = location.address
    except:
        #catches exception for the case where no value is returned
        #appends null value to column
        customer_relationship.loc[i,'destination_lat'] = ""
        customer_relationship.loc[i,'destination_long'] = ""
        customer_relationship.loc[i,'destination_address'] = ""

#print first rows as sample
customer_relationship.head()

In [ ]:

##Calculate_Distance between source and destination
from geopy import Point
from geopy.distance import distance
customer_relationship['s_point'] = customer_relationship.apply(lambda row: Point(latitude=row['source_lat'], 
                                                                           longitude=row['source_long']), axis=1)

customer_relationship['d_point'] = customer_relationship.apply(lambda row: Point(latitude=row['destination_lat'], 
                                                                           longitude=row['destination_long']), axis=1)


customer_relationship['distance_km'] = customer_relationship.apply(lambda row: distance(row['s_point'], row['d_point']).km if row['d_point'] is not None else float('nan'), axis=1)



customer_relationship.head()


In [ ]:

##Define activation dates as datetime
customer_relationship['Source_Activation_date'] = pd.to_datetime(customer_relationship['Source_Activation_date'])
customer_relationship['Destination_Activation_date'] = pd.to_datetime(customer_relationship['Destination_Activation_date'])

### Define conversion time in DAYS as difference between destination and source activation date

customer_relationship['conversion_time'] = (  customer_relationship['Destination_Activation_date'] - customer_relationship['Source_Activation_date']).dt.days





customer_relationship.head()


In [ ]:
#Fix invalid records

customer_relationship['s_point'] = customer_relationship['s_point'].astype(str)
customer_relationship['d_point'] = customer_relationship['d_point'].astype(str)




In [ ]:
##Define dataframe only for relationship indicators
s_d_indicators = customer_relationship[['Source_Customer_id',
                                     'Destination_Customer_id',
                                     'Source_Activation_date',
                                     'Destination_Activation_date',
                                     'conversion_time',
                                     'distance_km']]
s_d_indicators.head()



In [ ]:
###CREATE SQL TABLE

SQL_FULL_GEOCODED = """
CREATE TABLE [dbo].[Source_Destination_Full_Geocoded](
	[Source_Customer_id] [int] NULL,
	[Source_Full_name] [nvarchar](250) NULL,
	[Source_Customer_type] [nvarchar](250) NULL,
	[Source_Is_client_flg] [int] NULL,
	[Source_Country] [nvarchar](250) NULL,
	[Source_Region] [nvarchar](250) NULL,
	[Source_City] [nvarchar](250) NULL,
	[Source_Street] [nvarchar](250) NULL,
	[Source_Street_nr] [nvarchar](250) NULL,
	[Source_Activation_date] [datetime] NULL,
	[Source_Contact_phone] [nvarchar](250) NULL,
	[Source_Contact_email] [nchar](250) NULL,
	[Source_Acc_mgr_id] [int] NULL,
	[Source_Acc_mgr_name] [nchar](250) NULL,
	[Destination_Customer_id] [int] NULL,
	[Destination_Full_name] [nvarchar](250) NULL,
	[Destination_Customer_type] [nvarchar](250) NULL,
	[Destination_Is_client_flg] [int] NULL,
	[Destination_Country] [nvarchar](250) NULL,
	[Destination_Region] [nvarchar](250) NULL,
	[Destination_City] [nvarchar](250) NULL,
	[Destination_Street] [nvarchar](250) NULL,
	[Destination_Street_nr] [nvarchar](250) NULL,
	[Destination_Activation_date] [datetime] NULL,
	[Destination_Contact_phone] [nvarchar](250) NULL,
	[Destination_Contact_email] [nchar](250) NULL,
	[Destination_Acc_mgr_id] [int] NULL,
	[Destination_Acc_mgr_name] [nvarchar](250) NULL, 
	 ---new columns
 [Destination_full_address] [nvarchar](250) NULL,          
 [Source_full_address] [nvarchar](250) NULL,              
 [source_lat] [decimal](8,6) NULL,
 [source_long] [decimal](9,6) NULL,
 [source_address] [nvarchar](250) NULL,
 [destination_lat] [decimal](8,6) NULL,
 [destination_long] [decimal](9,6) NULL,
 [destination_address] [nvarchar](250) NULL,
 [s_point] [nvarchar](250) NULL,
 [d_point] [nvarchar](250) NULL,
 [distance_km] [decimal] NULL,
 [conversion_time] [int] NULL
 )"""

In [ ]:

##Import this CSV into SQL database
customer_relationship.to_csv (r'file/path', index = False) # place 'r' before the path name



In [ ]:
###Optional - you can export to CSV

#Connect to the database -- to be changed depending on configuration of the database

cnxn = pyodbc.connect(
    Trusted_Connection='Yes',
    Driver='{SQL Server Native Client 11.0}',
    Server='SERVER_NAME',
    Database='DATABASE_NAME'
)

cursor = cnxn.cursor()

customer.to_sql('Source_Destination_Full_Geocoded',con=engine, if_exists='append',index=False)

